# RCS CICAG workshop &mdash; KLIFS: making kinase structures work

- Kinase set: EGFR, ErbB2, SLK, BRAF?
  - Show how to use KLIFS Swagger API directly to fetch — Albert is doing this
  - Show how to use opencadd to fetch IFPs for kinase set
- Kinase KLIFS IDs by kinase names 
- Structures by kinase KLIFS IDs – select max. 2 liganded structures per kinase
- IFPs by structure KLIFS IDs
  - Show how to use kissim to encode pockets
- KiSSim fingerprints by structure KLIFS IDs
  - Calculate IFP- and KiSSim-based similarity
  - Show heatmap

In [1]:
try:
    import google.colab
    !pip install condacolab
    import condacolab
    condacolab.install()
    !mamba install -yq kissim
except ModuleNotFoundError:
    pass

## Generate a KLIFS Python client

In [2]:
from bravado.client import SwaggerClient

In [3]:
KLIFS_API_DEFINITIONS = "https://klifs.net/swagger/swagger.json"
KLIFS_CLIENT = SwaggerClient.from_url(KLIFS_API_DEFINITIONS, config={"validate_responses": False})

In [4]:
kinase_groups = KLIFS_CLIENT.Information.get_kinase_groups().response().result
print(kinase_groups)

['AGC', 'CAMK', 'CK1', 'CMGC', 'Other', 'STE', 'TK', 'TKL']


## OpenCADD-KLIFS

In [5]:
from opencadd.databases.klifs import setup_remote

session = setup_remote()

In [6]:
kinase_klifs_id = 406

In [7]:
structures = session.structures.by_kinase_klifs_id(kinase_klifs_id)
len(structures)

484

## KiSSim

In [8]:
from kissim.api import encode

In [9]:
structure_klifs_ids = [22, 222]

In [10]:
kissim_fingerprints = encode(structure_klifs_ids, n_cores=1)
kissim_fingerprints